In [1]:
from dotenv import load_dotenv
import os
from pprint import pprint

load_dotenv()
API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain_experimental.sql import SQLDatabaseChain

In [3]:
from langchain.prompts import PromptTemplate

TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
User asks about availability of making a booking, you must answer its questions using information about already boked time slots and masters working time
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}.

Here are some examples how you should think:
Question: Can I have a haircut on 11th of september at 17:00?
Thoughts: User want to check wether he can book a free slot for haircut to any master at 17:00. You should check if there any master available at 17:00 (he doesnt have any other bookings at that time)

Question: {input}"""

CUSTOM_PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=TEMPLATE
)

In [4]:
db = SQLDatabase.from_uri("sqlite:///BeautySalon.db")
llm = OpenAI(temperature=0, verbose=True)

In [5]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=CUSTOM_PROMPT)

In [6]:
TABLE_INFO = """
This is a basic schema for a beauty salon's database. 

Masters:
master_id: An integer that serves as the primary key for each master. It's auto-incremented.
name: A text field that stores the name of the master.
start_working_hour: A text field (in the format HH:MM) that indicates when the master starts their workday.
end_working_hour: A text field (in the format HH:MM) that indicates when the master ends their workday.

Services:
service_id: An integer that serves as the primary key for each service. It's auto-incremented.
name: A text field that describes the beauty service.
price: A real number that indicates the cost of the service.
duration_minutes: An integer that represents the duration of the service in minutes.

Clients:
client_id: An integer that serves as the primary key for each client. It's auto-incremented.
name: A text field that stores the name of the client.
phone: A text field that stores the client's phone number.

Bookings. Each booking have the master, client and service. If master is busy with the booking he cannot service another clients:
booking_id: An integer that serves as the primary key for each booking. It's auto-incremented.
client_id: A foreign key that references the client_id in the Clients table.
master_id: A foreign key that references the master_id in the Masters table.
service_id: A foreign key that references the service_id in the Services table.
date: A text field that stores the date of the booking.
start_time: A text field (in the format HH:MM) that indicates the start time of the booking.
end_time: A text field (in the format HH:MM) that indicates the end time of the booking. This is calculated based on the start_time and the duration of the service.
price: A real number that indicates the price of the booked service.
"""

In [7]:
dialect = "sqlite"

In [8]:
input_question = "Can I have a haircut on 11th of september at 17:00?"

In [9]:
db_chain.run(input=input_question, table_info=TABLE_INFO, dialect=dialect, query=input_question)



> Entering new SQLDatabaseChain chain...
Can I have a haircut on 11th of september at 17:00?
SQLQuery:SELECT * FROM Bookings b JOIN Masters m ON b.master_id = m.master_id WHERE b.date = '2023-09-11' AND b.start_time <= '17:00' AND b.end_time > '17:00'
SQLResult: 
Answer:No, there are no bookings at 17:00 on 11th of September.
> Finished chain.


'No, there are no bookings at 17:00 on 11th of September.'

In [10]:
db_chain.run("Change the time of booking on 5th of september to 11:00? I am Alice")



> Entering new SQLDatabaseChain chain...
Change the time of booking on 5th of september to 11:00? I am Alice
SQLQuery:SELECT * FROM Bookings WHERE date = '2023-09-05' AND client_id = (SELECT client_id FROM Clients WHERE name = 'Alice')
SQLResult: 
Answer:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-TJ1nPtWaKrWUu2YBxxnXGdTn on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..


KeyboardInterrupt: 

In [ ]:
db_chain.run("What are the non-booked windows for Emily on 5th of september? I need a haircut that takes around 60 minutes")

In [ ]:
db_chain.run("What is the salary for Palo Alto?")

In [12]:
db_chain.run("Please add a random row")



> Entering new SQLDatabaseChain chain...
Please add a random row
SQLQuery:INSERT INTO salaries VALUES (date('now'), 'City', 'Atherton', 'Administration', 'Assistant City Manager', 0, 0, '', 137826.0, 159550.0, 167872, 0, 0, 0, 0, 167872.0, 0, 11731.0, 0, 1686.0, 13417.0, '2% @ 55', 6921, 'San Mateo');
SQLResult: 
Answer:A new row has been added to the salaries table.
> Finished chain.


'A new row has been added to the salaries table.'

In [4]:
from typing import Optional, Type

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool


class SQLTool(BaseTool):
    name = "sql_tool"
    description = "Useful for when you need to answer questions about booking, reservations, appointment using SQL queries"

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return db_chain.run(query)

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [12]:
llm_chain = LLMChain(llm=llm, prompt=prompt)

# initialize the LLM tool
llm_tool = Tool(
    name='Language Model',
    func=llm_chain.run,
    description='use this tool for general purpose queries and logic'
)

NameError: name 'LLMChain' is not defined

In [13]:
tools = [SQLTool()]
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [14]:
agent.run("Is Emily available at 10pm on thursday?")



> Entering new AgentExecutor chain...
 I need to check the booking system
Action: sql_tool
Action Input: SELECT * FROM bookings WHERE name = 'Emily' AND time = '10pm' AND day = 'Thursday'

> Entering new SQLDatabaseChain chain...
SELECT * FROM bookings WHERE name = 'Emily' AND time = '10pm' AND day = 'Thursday'
SQLQuery:SELECT * FROM Appointments WHERE MasterName = 'Emily' AND StartTime = '22:00:00' AND AppointmentDate = date('now', '+3 days')
SQLResult: 
Answer:There are no bookings for Emily at 10pm on Thursday.
> Finished chain.

Observation: There are no bookings for Emily at 10pm on Thursday.
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-TJ1nPtWaKrWUu2YBxxnXGdTn on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-TJ1nPtWaKrWUu2YBxxnXGdTn on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

 I now know the final answer
Final Answer: No, Emily is not available at 10pm on Thursday.

> Finished chain.


'No, Emily is not available at 10pm on Thursday.'

In [5]:
from langchain.chains import create_extraction_chain

In [54]:
schema = {
    "properties": {
        "master_name": {"type": "string"},
        "beauty_service": {"type": "string"},
        "booking_date": {"type": "string"},
        "booking_time": {"type": "string"}
    },
    "required": ["service", "booking_date", "booking_time"]
}

In [131]:
llm = ChatOpenAI(openai_api_key=API_KEY, temperature=0)

In [65]:
ner_chain = create_extraction_chain(schema, llm)

In [66]:
class InsertNerTool(BaseTool):
    name = "insert_ner_tool"
    description = "Useful when user asks to make a booking for himself"

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return ner_chain.run(query)

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [84]:
tools = [InsertNerTool()]
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True,
    return_intermediate_steps=True
)

In [71]:
ner_chain.run("Can I have a haircut on 26th of september at 11:00?")

[{'booking_date': '26th of september', 'booking_time': '11:00'}]

In [ ]:
Extract and save the relevant entities mentionedin the following passage together with their properties.\n\nOnly extract the properties mentioned in the 'information_extraction' function.\n\nIf a property is not present and is not required in the function parameters, do not include it in the output

In [83]:
ner_chain.prompt

ChatPromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template="Extract and save the relevant entities mentionedin the following passage together with their properties.\n\nOnly extract the properties mentioned in the 'information_extraction' function.\n\nIf a property is not present and is not required in the function parameters, do not include it in the output.\n\nPassage:\n{input}\n", template_format='f-string', validate_template=True), additional_kwargs={})])

In [86]:
response = agent({"input": "I want to make my nails with Emily tomorrow at 10am"})



> Entering new AgentExecutor chain...
 I need to book an appointment
Action: insert_ner_tool
Action Input: Book an appointment with Emily at 10am tomorrow
Observation: [{'master_name': 'Emily', 'booking_time': '10am', 'booking_date': 'tomorrow'}]
Thought: I now know the final answer
Final Answer: Your appointment with Emily at 10am tomorrow has been booked.

> Finished chain.


In [81]:
response = agent.run("Hello, it's Natalie. Can I have my nails done tomorrow at 10am")

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-TJ1nPtWaKrWUu2YBxxnXGdTn on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..




> Entering new AgentExecutor chain...


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-TJ1nPtWaKrWUu2YBxxnXGdTn on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-TJ1nPtWaKrWUu2YBxxnXGdTn on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

 I need to book an appointment for Natalie
Action: insert_ner_tool
Action Input: Natalie, nails, tomorrow at 10am
Observation: [{'master_name': 'Natalie', 'beauty_service': 'nails', 'booking_date': 'tomorrow', 'booking_time': '10am'}]
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-TJ1nPtWaKrWUu2YBxxnXGdTn on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-TJ1nPtWaKrWUu2YBxxnXGdTn on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

KeyboardInterrupt: 

In [82]:
response = agent.run("Ann is free tomorrow? I want to make a pedicure")



> Entering new AgentExecutor chain...
 I need to find out if Ann is available for a pedicure tomorrow
Action: insert_ner_tool
Action Input: Ann's availability for a pedicure tomorrow
Observation: [{'beauty_service': 'pedicure', 'booking_date': 'tomorrow'}]
Thought: I now know that Ann is available for a pedicure tomorrow
Final Answer: Yes, Ann is available for a pedicure tomorrow.

> Finished chain.


In [69]:
type(response)

str

In [70]:
print(response)

Yes, you can have a haircut on 26th of September at 11:00.


In [173]:
template = """
Extract relevant entity mentioned in the following message together with it's properties. 
The message is about booking an appointment to a beauty master for a service at specified date and time. So the entity is booking. 

Only extract the following properties:
master_name - Name of the master to whom user wants to go
beauty_service - Name of the beauty service
booking_date - Date of booking. Transform to YYYY-MM-DD format. Current date is 2023-09-06
booking_time - Time of booking. Transform to HH:MM format

Map master names with this list:
- Alice Johnson
- Bob Smith
- Charlie Brown

Map service names with this list:
- Haircut & Styling
- Basic Facial
- Manicure (or nails)
- Pedicure
- Hair Coloring
- Eyebrow Threading
- Deep Tissue Massage

Output the result in the form of json with entity properties as keys

If some properties are missing then place null in their values

User input: {input_text}
Result of retrieval:
"""

In [174]:
prompt = PromptTemplate(template=template, input_variables=["input_text"])

In [175]:
ner_new_chain = LLMChain(prompt=prompt, llm=llm)

In [176]:
result = ner_new_chain.run({"input_text": "Hello, it's Natalie. Can I have my nails done 26th of september at 10am"})

In [180]:
from json import loads

In [181]:
loads(result)

{'master_name': None,
 'beauty_service': 'Manicure',
 'booking_date': '2023-09-26',
 'booking_time': '10:00'}

In [170]:
pprint(ner_new_chain.run({"input_text": "Hello, it's Natalie. Can I have my nails done 26th of september at 10am"}))

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-TJ1nPtWaKrWUu2YBxxnXGdTn on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-TJ1nPtWaKrWUu2YBxxnXGdTn on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

('{\n'
 '  "master_name": null,\n'
 '  "beauty_service": "Manicure",\n'
 '  "booking_date": "2023-09-26",\n'
 '  "booking_time": "10:00"\n'
 '}')


In [171]:
pprint(ner_new_chain.run({"input_text": "Can I have a haircut tomorrow at 11:00 with Bob?"}))

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-TJ1nPtWaKrWUu2YBxxnXGdTn on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-TJ1nPtWaKrWUu2YBxxnXGdTn on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

('{\n'
 '  "master_name": "Bob Smith",\n'
 '  "beauty_service": "Haircut & Styling",\n'
 '  "booking_date": "2023-09-07",\n'
 '  "booking_time": "11:00"\n'
 '}')


In [177]:
pprint(ner_new_chain.run({"input_text": "Ann is free tomorrow? I want to make a pedicure"}))

('{\n'
 '  "master_name": null,\n'
 '  "beauty_service": "Pedicure",\n'
 '  "booking_date": "2023-09-07",\n'
 '  "booking_time": null\n'
 '}')


In [101]:
from langchain import LLMChain